In [1]:
# learn the physician policy - ie, pi(a|s)

In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import pickle

In [20]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [23]:
train_data = pd.read_csv('../data_o/rl_train_data_final_cont.csv')

In [24]:
val_data = pd.read_csv('../data_o/rl_val_data_final_cont.csv')

In [25]:
test_data = pd.read_csv('../data_o/rl_test_data_final_cont.csv')

In [26]:
# Extract features (state vector) and labels (action taken) out of the dataframe for train 
# and val sets
def preproc(df_in, iv_bins = 5):
    df = df_in.copy()
    actions_raw = df[['iv_input', 'vaso_input']].values
    keep_arr = np.loadtxt('../data/state_features.txt', dtype=str)
    df = df[keep_arr]
    actions_proc = (iv_bins*actions_raw[:, 0] + actions_raw[:, 1]).astype(int)
    #hist = np.histogram(actions_proc, 25)
    actions_proc = pd.get_dummies(actions_proc).values
    #print(hist) 
    return df.values, actions_proc

In [27]:
def batch_sample(batch_size, features, labels):
    idx = np.random.choice(np.arange(len(features)), batch_size)
    return (np.vstack(features[idx]), np.vstack(labels[idx]))

In [28]:
train_feat, train_labels = preproc(train_data)
val_feat, val_labels = preproc(val_data)
test_feat, test_labels = preproc(test_data)

In [29]:
feature_length = len(train_feat[0])
batch_size = 64
num_actions = 25
num_steps = 70000

In [30]:
# todo - reduce network size
class PolicyModel():
    def __init__(self):
        self.input_feat = tf.placeholder(tf.float32, shape = [None, feature_length])
        self.labels = tf.placeholder(tf.float32, shape = [None, num_actions])
        self.phase = tf.placeholder(tf.bool)
        
        self.fc_1 = tf.contrib.layers.fully_connected(self.input_feat, 64, activation_fn=tf.nn.relu) #64
        self.bn_1 = tf.contrib.layers.batch_norm(self.fc_1, center=True, scale=True, is_training=self.phase)
#         self.fc_2 = tf.contrib.layers.fully_connected(self.bn_1 , 256, activation_fn=tf.nn.relu)    
#         self.bn_2 = tf.contrib.layers.batch_norm(self.fc_2 , center=True, scale=True, is_training=self.phase)
#         L2_drop_2 = tf.nn.dropout(self.bn_2,0.8)
#         self.fc_3 = tf.contrib.layers.fully_connected(L2_drop_2, 128, activation_fn=tf.nn.relu)
#         self.bn_3 = tf.contrib.layers.batch_norm(self.fc_3, center=True, scale=True, is_training=self.phase)
#         L2_drop_3 = tf.nn.dropout(self.fc_3,0.8)
        self.fc_4 = tf.contrib.layers.fully_connected(self.bn_1 , 64, activation_fn=tf.nn.relu) #64
        self.bn_4 = tf.contrib.layers.batch_norm(self.fc_4, center=True, scale=True, is_training=self.phase)
        
        self.logits = tf.contrib.layers.fully_connected(self.bn_4 , num_actions, activation_fn=None)
        self.output = tf.nn.softmax(self.logits)
        self.reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        self.reg_constant = 0.1 
        
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.labels, 1), tf.argmax(self.output, 1)),'float32'))
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.labels)) + self.reg_constant*sum(self.reg_losses)

        
        self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
            self.train_step = tf.train.AdamOptimizer().minimize(self.loss)

In [31]:
# Prints out accuracy on the relevant dataset and returns the policy. 
# This is the probability of taking each action in the action space from that state

def get_policy(dataset,sess, mdl):

    if dataset == 'train':
        features, labels = train_feat,train_labels
    elif dataset == 'val':
        features, labels = val_feat,val_labels
    elif dataset == 'test':
        features, labels = test_feat,test_labels

    
    op = np.zeros((len(features), num_actions))
    total_acc = 0
    total_loss = 0
    j = 0
    while (j < len(features)):
        feat = None
        lbls = None
        if len(features) - j < batch_size:
            feat = features[j:-1]
            lbls = labels[j:-1]
        else:
            feat = features[j:j+batch_size]
            lbls = labels[j:j+batch_size]
        feat = feat.reshape(len(feat), feature_length)
        lbls = lbls.reshape(len(lbls), num_actions)
        if j%10000 == 0: print('Processing val set indx: ', j )
        softmax, accuracy, loss = sess.run([mdl.output, mdl.accuracy, mdl.loss], feed_dict={mdl.input_feat : feat, mdl.phase: 0, mdl.labels: lbls, mdl.phase: 0})
        total_acc += accuracy
        op[j:j+len(feat)] = softmax
        if len(features) - j < batch_size:
            j = len(features)
        else: j+=batch_size
        final_acc = total_acc/(len(op)/batch_size)
        total_loss += loss
    return op, final_acc, total_loss
    

In [32]:
def train():
    tf.reset_default_graph()
    mdl = PolicyModel()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  # Don't use all GPUs 
    config.allow_soft_placement = True  # Enable manual control
    init = tf.global_variables_initializer()
    with tf.Session(config=config) as sess:
        sess.run(init)
        net_loss = 0
        net_accuracy = 0.0
        print('Starting training!')
        for i in range(num_steps):
            feat, labels = batch_sample(batch_size, train_feat, train_labels)
            
            _, loss, accuracy = sess.run([mdl.train_step, mdl.loss, mdl.accuracy], feed_dict={mdl.input_feat : feat, mdl.labels: labels, mdl.phase: 1})
            
            net_loss += loss
            net_accuracy += accuracy
            if i % 1000 == 0 and i > 0:
                av_loss = net_loss/1000.0
                av_accuracy = net_accuracy/1000.0
                print("Step: ", i, "Average loss is: ", av_loss, "Average accuracy is: ", av_accuracy)
                net_loss = 0.0
                net_accuracy = 0.0
            
            if i % 10000 == 0:
                print ("Test on validation set")
                _, val_acc, val_loss = get_policy('val', sess, mdl)
                print('Val set accuracy, loss: ', val_acc, val_loss)
                
        # Commented out for now
        # train_policy, train_acc = get_policy('train')
        print ("Finished, getting final accuracy")
        val_policy, val_acc, val_loss = get_policy('val', sess, mdl)
        test_policy, test_acc, _ = get_policy('test',sess, mdl)
    print('Val set accuracy, loss: ', val_acc, val_loss)
    print('Test set accuracy: ', test_acc)

    return val_policy, test_policy


In [14]:
val_policy, test_policy = train()



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.





Starting training!
Test on validation set
Processing val set indx:  0
Val set accuracy, loss:  0.004026893898536656 1447.8720092773438
Step:  1000 Average loss is:  2.004774188041687 Average accuracy is:  0.33409375
Step:  2000 Average loss is:  1.7374808579683303 Average accuracy is:  0.359015625
Step:  3000 Average loss is:  1.7111755373477935 Average accuracy i

Val set accuracy, loss:  0.3655724541395951 732.9982464313507
Step:  61000 Average loss is:  1.5576342287063598 Average accuracy is:  0.395046875
Step:  62000 Average loss is:  1.5581345125436783 Average accuracy is:  0.3981875
Step:  63000 Average loss is:  1.5548220794200898 Average accuracy is:  0.400296875
Step:  64000 Average loss is:  1.5554102017879485 Average accuracy is:  0.3971875
Step:  65000 Average loss is:  1.5535350737571716 Average accuracy is:  0.397796875
Step:  66000 Average loss is:  1.5564927599430085 Average accuracy is:  0.39540625
Step:  67000 Average loss is:  1.5572698106765748 Average accuracy is:  0.398078125
Step:  68000 Average loss is:  1.5583278810977936 Average accuracy is:  0.395484375
Step:  69000 Average loss is:  1.5498424896001817 Average accuracy is:  0.39925
Finished, getting final accuracy
Processing val set indx:  0
Processing val set indx:  0
Processing val set indx:  40000
Val set accuracy, loss:  0.3707858435618077 710.6563432216644
Test set

In [39]:
#  save the learned policy as a numpy array with the columns as icustayid, bloc, iv input, vaso input,
#  action index (of 25), and probability distribution over actions ( this is 25 columns)

In [15]:
v_data = val_data[['icustayid', 'bloc', 'iv_input', 'vaso_input']].values
val_actions = (5*val_data['iv_input'].values + val_data['vaso_input']).values.astype(int)
val_pickle = np.concatenate((v_data, val_actions.reshape(len(val_actions), 1), val_policy), axis = 1)

In [16]:
t_data = test_data[['icustayid', 'bloc', 'iv_input', 'vaso_input']].values
test_actions = (5*test_data['iv_input'].values + test_data['vaso_input']).values.astype(int)
test_pickle = np.concatenate((t_data, test_actions.reshape(len(test_actions), 1), test_policy), axis = 1)

In [17]:
with open(r"val_policy.p", "wb") as f:
    pickle.dump(val_pickle, f)

In [18]:
with open(r"test_policy.p", "wb") as f:
    pickle.dump(test_pickle, f)